In [1]:
!wget "https://www.machinehack.com/wp-content/uploads/2020/04/Insurance_Churn_ParticipantsData.zip"

--2020-04-26 18:44:54--  https://www.machinehack.com/wp-content/uploads/2020/04/Insurance_Churn_ParticipantsData.zip
Resolving www.machinehack.com (www.machinehack.com)... 107.180.71.20
Connecting to www.machinehack.com (www.machinehack.com)|107.180.71.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1346633 (1.3M) [application/zip]
Saving to: ‘Insurance_Churn_ParticipantsData.zip’

Insurance_Churn_Par 100%[===================>]   1.28M  2.35MB/s    in 0.5s    

2020-04-26 18:44:55 (2.35 MB/s) - ‘Insurance_Churn_ParticipantsData.zip’ saved [1346633/1346633]



In [2]:
!unzip Insurance_Churn_ParticipantsData

Archive:  Insurance_Churn_ParticipantsData.zip
   creating: Insurance_Churn_ParticipantsData/
  inflating: Insurance_Churn_ParticipantsData/Test.csv  
  inflating: __MACOSX/Insurance_Churn_ParticipantsData/._Test.csv  
  inflating: Insurance_Churn_ParticipantsData/sample_submission.xlsx  
  inflating: __MACOSX/Insurance_Churn_ParticipantsData/._sample_submission.xlsx  
  inflating: Insurance_Churn_ParticipantsData/Train.csv  
  inflating: __MACOSX/Insurance_Churn_ParticipantsData/._Train.csv  


In [0]:
import pandas as pd
import numpy as np

In [4]:
train_df = pd.read_csv('Insurance_Churn_ParticipantsData/Train.csv')
train_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,labels
0,-0.276515,-0.424429,1.344997,-0.012283,0.076230,1.076648,0.182198,3,0,1,0,0,0,0,10,2,1
1,0.853573,0.150991,0.503892,-0.979179,-0.569351,-0.411453,-0.251940,4,1,2,0,1,0,0,0,3,0
2,0.947747,-0.173832,1.825628,-0.703478,0.076230,-0.411453,-0.251940,6,1,2,0,0,0,0,5,3,0
3,0.853573,-0.381404,0.984523,-0.039464,-0.569351,-0.411453,-0.251940,4,0,2,0,1,0,0,5,3,0
4,1.324443,1.590527,-1.178318,-0.097711,-0.246560,-0.411453,-0.251940,0,1,1,0,0,0,0,8,3,0


In [5]:
x = train_df.drop(['labels'], axis=1)
x.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,-0.276515,-0.424429,1.344997,-0.012283,0.076230,1.076648,0.182198,3,0,1,0,0,0,0,10,2
1,0.853573,0.150991,0.503892,-0.979179,-0.569351,-0.411453,-0.251940,4,1,2,0,1,0,0,0,3
2,0.947747,-0.173832,1.825628,-0.703478,0.076230,-0.411453,-0.251940,6,1,2,0,0,0,0,5,3
3,0.853573,-0.381404,0.984523,-0.039464,-0.569351,-0.411453,-0.251940,4,0,2,0,1,0,0,5,3
4,1.324443,1.590527,-1.178318,-0.097711,-0.246560,-0.411453,-0.251940,0,1,1,0,0,0,0,8,3


In [6]:
y = train_df.labels

test_df = pd.read_csv('Insurance_Churn_ParticipantsData/Test.csv')
test_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,0.571051,0.406843,0.984523,0.011016,-0.569351,-0.411453,-0.251940,0,1,1,0,0,0,0,11,3
1,-1.124080,-0.166935,0.503892,-0.322932,0.721811,0.547323,0.182198,0,2,1,0,0,0,0,5,1
2,0.476877,0.145079,-0.577529,-0.691828,-0.246560,-0.411453,-0.251940,0,1,1,0,0,0,0,1,3
3,1.606965,-0.447419,1.825628,-0.983062,7.177616,-0.411453,-0.251940,1,1,0,0,1,0,0,5,3
4,-0.935732,-0.364653,-1.178318,-0.322932,0.076230,-0.411453,-0.251940,8,2,1,0,1,0,2,8,3


In [19]:
type(y)

pandas.core.series.Series

In [21]:
y.value_counts()

0    29941
1     3967
Name: labels, dtype: int64

scale_pos_weight = total_negative_examples / total_positive_examples

In [64]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

model = XGBClassifier(scale_pos_weight=3, n_estimators=300, base_score=0.9)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

scores = cross_val_score(model, x_norm, y, scoring='f1', cv=cv, n_jobs=-1)
print('Mean f1 Score: %.5f' % mean(scores))

Mean f1 Score: 0.62073


In [0]:
#3,60548,59852
#md 5,62337,58758

In [0]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(x)
x_norm = transformer.transform(x)

predict_on = transformer.transform(test_df)

In [56]:
import xgboost as xgb

clf = xgb.XGBClassifier(scale_pos_weight=3)
clf.fit(x_norm, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=3, seed=None,
              silent=None, subsample=1, verbosity=1)

In [57]:
clf.score(x_norm, y)

0.8938893476465731

In [0]:
z = clf.predict(predict_on)

In [59]:
submit = pd.read_excel('Insurance_Churn_ParticipantsData/sample_submission.xlsx')
submit.head()

,labels
0,1
1,1
2,1
3,1
4,1


In [0]:
submit.labels = z
#submit.head()

In [0]:
submit.to_excel('submit-md5.xlsx', index=False)